In [1]:
import pandas as pd

# CSV 파일 읽기
money = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(추정매출-상권).csv')  # DataFrame 객체로 로드
human = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(길단위인구-상권).csv')
map_sang = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(영역-상권).csv')
coordinate = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\위도경도.csv')
store = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\점포.csv')
jong = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\업종별대시보드(before_전처리).csv')

In [2]:
#필요한 컬럼만 선택하기
money = money[['기준_년분기_코드','상권_구분_코드_명','상권_코드','상권_코드_명','당월_매출_금액','서비스_업종_코드_명']]
human = human[['기준_년분기_코드','상권_코드','총_유동인구_수']]
map_sang = map_sang[['상권_코드','자치구_코드_명','행정동_코드_명']]
coordinate = coordinate[['상권_코드','경도1','위도1']]
store = store[['기준_년분기_코드', '상권_코드','점포_수']]
jong = jong[['서비스_업종_코드_명','업종_중분류']]
jong = jong.drop_duplicates(subset=['서비스_업종_코드_명', '업종_중분류'], keep='first')

In [3]:
# 추정매출과 길단위인구 조인 1step
join_1step = pd.merge(money, human, on=['기준_년분기_코드', '상권_코드'], how='inner')

In [4]:
# 1차에서 조인 되지 못한 상권코드
unmatched_from_df1 = money.merge(human, on=['기준_년분기_코드', '상권_코드'], how='left', indicator=True)
unmatched_code = unmatched_from_df1[unmatched_from_df1['_merge'] == 'left_only']

In [5]:
# 1step에서 조인 한것과 영역 데이터 조인(2차조인)
join_2step = pd.merge(join_1step, map_sang, on=['상권_코드'], how='inner')

In [6]:
#2차에서 조인 되지 못한 상권코드
unmatched_from_df1 = join_1step.merge(map_sang, on=['상권_코드'], how='left', indicator=True)
unmatched_code2 = unmatched_from_df1[unmatched_from_df1['_merge'] == 'left_only']

In [7]:
# '기준_년분기_코드' 값을 년월로 변환하는 함수 정의
def convert_to_date(code):
    year = int(str(code)[:4])  # 연도 추출
    quarter = int(str(code)[-1])  # 분기 추출
    month = (quarter - 1) * 3 + 1  # 분기에 따라 시작 월 계산
    return pd.Timestamp(year=year, month=month, day=1)
# 변환 적용
join_2step['기준_년분기_코드_날짜'] = join_2step['기준_년분기_코드'].apply(convert_to_date)

In [8]:
join_3step = join_2step[join_2step['기준_년분기_코드_날짜'] >= pd.Timestamp(year=2024, month=1, day=1)]

In [9]:
# 데이터 타입 최적화
coordinate['상권_코드'] = coordinate['상권_코드'].astype('category')
join_3step['상권_코드'] = join_3step['상권_코드'].astype('category')


C:\Users\m\AppData\Local\Temp\ipykernel_19296\1505405958.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  join_3step['상권_코드'] = join_3step['상권_코드'].astype('category')


In [10]:
# 2step에서 조인 한것과 영역 데이터 조인(3차조인)
join_4step = pd.merge(join_3step, coordinate, on=['상권_코드'], how='inner')

In [11]:
#3차에서 조인 되지 못한 상권코드
unmatched_from_df2 = join_3step.merge(coordinate, on=['상권_코드'], how='left', indicator=True)
unmatched_code3 = unmatched_from_df2[unmatched_from_df2['_merge'] == 'left_only']

In [12]:
# 중복 제거
join_4step = join_4step.drop_duplicates()

In [13]:
# 3step에서 조인 한것과 점포수 데이터 조인(4차조인)
join_5step = pd.merge(join_4step, store, on=['기준_년분기_코드', '상권_코드'], how='inner')

In [14]:
# 3step에서 조인 한것과 점포수 데이터 조인(4차조인)
join_final = pd.merge(join_4step, store, on=['기준_년분기_코드', '상권_코드'], how='inner')
join_final = pd.merge(join_final, jong, on=['서비스_업종_코드_명'], how='inner')

In [15]:
join_final

,기준_년분기_코드,상권_구분_코드_명,상권_코드,상권_코드_명,당월_매출_금액,서비스_업종_코드_명,총_유동인구_수,자치구_코드_명,행정동_코드_명,기준_년분기_코드_날짜,경도1,위도1,점포_수,업종_중분류
0,20241,골목상권,3110242,면목5동주민센터,477000000.0,컴퓨터및주변장치판매,543331,중랑구,면목5동,2024-01-01,127.048893,37.585913,3,소매
1,20241,골목상권,3110242,면목5동주민센터,477000000.0,컴퓨터및주변장치판매,543331,중랑구,면목5동,2024-01-01,127.048893,37.585913,0,소매
2,20241,골목상권,3110242,면목5동주민센터,477000000.0,컴퓨터및주변장치판매,543331,중랑구,면목5동,2024-01-01,127.048893,37.585913,10,소매
3,20241,골목상권,3110242,면목5동주민센터,477000000.0,컴퓨터및주변장치판매,543331,중랑구,면목5동,2024-01-01,127.048893,37.585913,4,소매
4,20241,골목상권,3110242,면목5동주민센터,477000000.0,컴퓨터및주변장치판매,543331,중랑구,면목5동,2024-01-01,127.048893,37.585913,2,소매
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666464,20241,발달상권,3120007,북촌(안국역),192542931.0,치킨전문점,705408,종로구,가회동,2024-01-01,126.991088,37.580397,77,음식/음료
666465,20241,발달상권,3120007,북촌(안국역),192542931.0,치킨전문점,705408,종로구,가회동,2024-01-01,126.991088,37.580397,3,음식/음료
666466,20241,발달상권,3120007,북촌(안국역),192542931.0,치킨전문점,705408,종로구,가회동,2024-01-01,126.991088,37.580397,6,음식/음료
666467,20241,발달상권,3120007,북촌(안국역),192542931.0,치킨전문점,705408,종로구,가회동,2024-01-01,126.991088,37.580397,9,음식/음료


In [16]:
join_final = join_final.drop_duplicates(keep='first')

In [39]:
#최종 데이터 저장
join_final.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\종합_대시보드데이터_v0.1.csv', index=False, encoding='utf-8-sig')
unmatched_code.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\유동인구없는_상권코드.csv', index=False, encoding='utf-8-sig')
unmatched_code3.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\좌표없는_상권코드.csv', index=False, encoding='utf-8-sig')

## 전분기 대비 증감 컬럼 구하기

In [17]:
# 기준_년분기_코드 정렬 (필요 시)
data = join_final.sort_values(by='기준_년분기_코드')

# 그룹화하여 합계계산
exclude_cols = ['점포_수', '총_유동인구_수'] # 증감에 사용할 컬럼 분리
group_cols = [col for col in data.columns if col not in exclude_cols]
data = data.groupby(group_cols)[exclude_cols].sum().reset_index()

In [18]:
# 기준_년분기_코드를 숫자로 변환
data['기준_년분기_코드'] = data['기준_년분기_코드'].astype(int)

# 연도와 분기를 분리
data['연도'] = data['기준_년분기_코드'] // 10  # 연도 추출
data['분기'] = data['기준_년분기_코드'] % 10   # 분기 추출

# 전분기 계산: 분기가 1인 경우 전년도 4분기로 이동
data['전분기_연도'] = data['연도']
data['전분기_분기'] = data['분기'] - 1
data.loc[data['전분기_분기'] == 0, '전분기_연도'] -= 1  # 전년도 처리
data.loc[data['전분기_분기'] == 0, '전분기_분기'] = 4   # 전년도 4분기로 설정

# 전분기의 기준_년분기_코드 생성 (연도 * 10 + 분기)
data['전분기_코드'] = data['전분기_연도'] * 10 + data['전분기_분기']

# 셀프 조인: 현재 데이터와 전 분기의 데이터를 기준으로 병합
merged_data = pd.merge(
    data,
    data[['상권_코드', '기준_년분기_코드', '점포_수', '총_유동인구_수']],
    left_on=['상권_코드', '전분기_코드'],
    right_on=['상권_코드', '기준_년분기_코드'],
    how='left',
    suffixes=('', '_전분기')
)

# 전분기 대비 차이 계산
merged_data['점포_수_전분기대비'] = merged_data['점포_수'] - merged_data['점포_수_전분기']
merged_data['총유동인구수_전분기대비'] = merged_data['총_유동인구_수'] - merged_data['총_유동인구_수_전분기']

In [19]:
#조인 할떄 발생한 중복제거
merged_data = merged_data.drop_duplicates(keep='first')

In [20]:
#전분기 컬럼이 포함된 v0.2 데이터 저장장
merged_data.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\종합_데시보드데이터_v0.2.csv', index=False, encoding='utf-8-sig')

## 업종의 점포수, 상권의 유동인구수 증감률 별도 계산 및 데이터 분리

In [21]:
store_group_sum = merged_data.groupby(['기준_년분기_코드_날짜', '서비스_업종_코드_명'])[['점포_수', '점포_수_전분기']].sum().reset_index()
map_group_sum = merged_data.groupby(['기준_년분기_코드_날짜', '상권_코드_명'])[['총_유동인구_수', '총_유동인구_수_전분기']].sum().reset_index()

In [22]:
# 증감율 계산 함수 정의
def calculate_growth_rate(current, previous):
    if previous == 0 or pd.isna(previous):  # 이전 값이 0 또는 NaN일 경우 증감율은 0으로 처리
        return 0
    else:
        return ((current - previous) / previous) * 100

# 증감율 계산
store_group_sum['점포수_증감율'] = store_group_sum.apply(lambda row: calculate_growth_rate(row['점포_수'], row['점포_수_전분기']), axis=1)
map_group_sum['유동인구수_증감율'] = map_group_sum.apply(lambda row: calculate_growth_rate(row['총_유동인구_수'], row['총_유동인구_수_전분기']), axis=1)

In [23]:
# 증감 상태 컬럼 추가
def determine_change_status(growth_rate):
    if growth_rate > 0:
        return '증가'
    elif growth_rate < 0:
        return '감소'
    else:
        return '-'

store_group_sum['증감상태'] = store_group_sum['점포수_증감율'].apply(determine_change_status)
map_group_sum['증감상태'] = map_group_sum['유동인구수_증감율'].apply(determine_change_status)

In [32]:
store_join = []
store_join = merged_data[['업종_중분류', '서비스_업종_코드_명']]
store_join =  store_join.drop_duplicates(subset=['업종_중분류', '서비스_업종_코드_명'], keep='first')
store_group_sum = pd.merge(store_group_sum, store_join, on='서비스_업종_코드_명', how='inner')

In [34]:
map_join = []
map_join = merged_data[['자치구_코드_명', '상권_코드_명']]
map_join =  map_join.drop_duplicates(subset=['자치구_코드_명', '상권_코드_명'], keep='first')
map_group_sum = pd.merge(map_group_sum, map_join, on='상권_코드_명', how='inner')

In [36]:
# 증감율 데이터 저장
store_group_sum.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\업종별_증감율_V0.1.csv', index=False, encoding='utf-8-sig')
map_group_sum.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\상권별_증감율_V0.1.csv', index=False, encoding='utf-8-sig')

In [35]:
map_group_sum

,기준_년분기_코드_날짜,상권_코드_명,총_유동인구_수,총_유동인구_수_전분기,유동인구수_증감율,증감상태,자치구_코드_명
0,2024-01-01,4.19민주묘지역 2번,12254100,0.0,0.000000,-,강북구
1,2024-01-01,63빌딩,2423226,0.0,0.000000,-,영등포구
2,2024-01-01,DMC(디지털미디어시티),317709630,0.0,0.000000,-,마포구
3,2024-01-01,GS강동자이아파트,81290314,0.0,0.000000,-,강동구
4,2024-01-01,G타워,8246040,0.0,0.000000,-,구로구
...,...,...,...,...,...,...,...
2971,2024-04-01,후암동주민센터,481223358,499988874.0,-3.753187,감소,용산구
2972,2024-04-01,후암시장,13880256,11938524.0,16.264423,증가,용산구
2973,2024-04-01,휘문고교사거리,30644341,34877752.0,-12.137855,감소,강남구
2974,2024-04-01,흑리단길,15324582,13281528.0,15.382673,증가,동작구
